# This notebook is for downloading financial timeseries from MetaTrader ver.5

In [1]:
import pandas as pd
import plotly.express as px
import MetaTrader5 as mt
from datetime import datetime
import pytz

In [1]:
# initialize MetaTrader 5
mt.initialize(login="your login number", server="MetaQuotes-Demo", password="your password")
if mt.initialize():
    print ("Initialization succesful.","\n",
           "MetaTrader5 package author: ",mt.__author__,"\n",
           "MetaTrader5 package version: ",mt.__version__)

if not mt.initialize():
    print("initialize() failed!!!, error code =",mt.last_error())
    quit()

Initialization succesful.


In [7]:
# import the 'pandas' module for displaying data obtained in the tabular form
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display
 
# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2000, 1, 1, tzinfo=timezone)
utc_to = datetime(2022, 1, 1, hour = 13, tzinfo=timezone)

# get XAUUSD H1 bars starting from 01.10.2020 in UTC time zone
rates = mt.copy_rates_range("XAUUSD", mt.TIMEFRAME_D1, utc_from, utc_to)
display (rates)

array([(1086912000,  384.  ,  384.8 ,  382.8 ,  384.1 ,   303, 0, 0),
       (1087171200,  384.3 ,  385.8 ,  381.8 ,  382.8 ,  1954, 0, 0),
       (1087257600,  382.8 ,  388.8 ,  381.1 ,  388.6 ,  1995, 0, 0), ...,
       (1640736000, 1806.56, 1807.71, 1789.47, 1803.9 , 74622, 5, 0),
       (1640822400, 1803.71, 1817.21, 1796.12, 1815.08, 71293, 5, 0),
       (1640908800, 1814.64, 1828.57, 1814.2 , 1828.27, 59072, 5, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [13]:
rates_frame = pd.DataFrame(rates)
#Change index
rates_frame = rates_frame.set_index ('time')
# convert time in seconds into the datetime format
rates_frame.index=pd.to_datetime(rates_frame.index, unit='s').to_period ('D') 
# rates_frame ['day_of_year'] = rates_frame.index.dayofyear
# rates_frame ['day_of_week'] = rates_frame.index.dayofweek
# rates_frame ['hour_of_day'] = rates_frame.index.hour
rates_frame.reset_index (inplace=True)
display (rates_frame)

,time,open,high,low,close,tick_volume,spread,real_volume
0,2004-06-11,384.00,384.80,382.80,384.10,303,0,0
1,2004-06-14,384.30,385.80,381.80,382.80,1954,0,0
2,2004-06-15,382.80,388.80,381.10,388.60,1995,0,0
3,2004-06-16,387.10,389.80,382.60,383.80,2106,0,0
4,2004-06-17,383.60,389.30,383.00,387.60,1638,0,0
...,...,...,...,...,...,...,...,...
4501,2021-12-27,1805.97,1813.47,1802.92,1811.82,66267,5,0
4502,2021-12-28,1811.55,1820.18,1805.05,1806.13,65449,5,0
4503,2021-12-29,1806.56,1807.71,1789.47,1803.90,74622,5,0
4504,2021-12-30,1803.71,1817.21,1796.12,1815.08,71293,5,0


In [14]:
df = rates_frame.copy ().drop(['real_volume' , 'spread'] , axis=1)
display (df) #use for input

,time,open,high,low,close,tick_volume
0,2004-06-11,384.00,384.80,382.80,384.10,303
1,2004-06-14,384.30,385.80,381.80,382.80,1954
2,2004-06-15,382.80,388.80,381.10,388.60,1995
3,2004-06-16,387.10,389.80,382.60,383.80,2106
4,2004-06-17,383.60,389.30,383.00,387.60,1638
...,...,...,...,...,...,...
4501,2021-12-27,1805.97,1813.47,1802.92,1811.82,66267
4502,2021-12-28,1811.55,1820.18,1805.05,1806.13,65449
4503,2021-12-29,1806.56,1807.71,1789.47,1803.90,74622
4504,2021-12-30,1803.71,1817.21,1796.12,1815.08,71293


## Save the data to CSV file

In [15]:
df.to_csv ('D:\XAU_D.csv')

# Read account data

In [14]:
# set the symbol and volume
symbol = "XAUUSD"
volume = 0.1

# get the current bid and ask prices
bid = mt.symbol_info_tick(symbol).bid
ask = mt.symbol_info_tick(symbol).ask
spread = ask - bid
print(f"Symbol: {symbol}\nSpread: {round(spread , 2)} points")

info = mt.account_info()
balance = info.balance
equity = info.equity
print (f"Balance: {balance}\nEquity: {equity}")

Asset_info = pd.DataFrame ({
    'Spread': [spread],
    'Ask price' : [ask],
    'Bid price' : [bid]
    },
index = [symbol]
)
Asset_info["spread_new"] = Asset_info["Ask price"].apply(lambda x: 2*(x - Asset_info['Bid price']))
print (Asset_info)
#display (spread_new)


        Spread  Ask price  Bid price  spread_new
XAUUSD    0.53    2011.31    2010.78        1.06


# This section is useful if you want to send orders (Buy/Sell) to MT% terminal via Python.

In [ ]:
# prepare the buy request structure
buy_request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt5.ORDER_TYPE_BUY,
    "price": ask,
    "deviation": 10,
    "magic": 100,
    "comment": "buy by python",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

# send a trading request
result = mt5.order_send(buy_request)

# check the execution result
print("Buy order result:", result)

# get the ticket number of the opened position
ticket = result.order

# wait for 10 seconds
# import time
# time.sleep(10)

# prepare the sell request structure
# sell_request = {
#     "action": mt5.TRADE_ACTION_DEAL,
#     "position": ticket,
#     "symbol": symbol,
#     "volume": volume,
#     "type": mt5.ORDER_TYPE_SELL,
#     "price": bid,
#     "deviation": 10,
#     "magic": 100,
#     "comment": "sell by python",
#     "type_time": mt5.ORDER_TIME_GTC,
#     "type_filling": mt5.ORDER_FILLING_RETURN,
# }

# # send a trading request
# result = mt5.order_send(sell_request)

# # check the execution result
# print("Sell order result:", result)

# shut down connection to MetaTrader 5
# mt5.shutdown()
